## Usando RAG para Docs de condomínio


In [ ]:
# RAG para Documentos de Condomínio - Versão Otimizada

import os
import glob
from dotenv import load_dotenv
import gradio as gr

# Imports LangChain
from langchain.document_loaders import DirectoryLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.document_loaders import UnstructuredFileLoader
from langchain.prompts import PromptTemplate



In [ ]:
# ============================================================================
# 1. CONFIGURAÇÃO INICIAL
# ============================================================================

# Configuração de modelos e diretórios
MODEL = "gpt-4o-mini"
DB_NAME = "vector_db"

# Carregamento de variáveis de ambiente
load_dotenv(override=True)
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')


In [ ]:
# 1. Primeiro, verifique se o caminho está correto
BASE_DIR = os.path.abspath(os.path.join(os.getcwd(), "../0_base_conhecimento"))
CACHE_DIR = os.path.join(BASE_DIR, "processed_docs_cache")

knowledgebase_path = CACHE_DIR
print(f"Verificando o diretório: {knowledgebase_path}")
print(f"Conteúdo do diretório: {os.listdir(knowledgebase_path)}")

# 2. Modifique para procurar tanto em processed_docs_cache quanto em suas subpastas
folders = glob.glob(os.path.join(knowledgebase_path, "*")) + [knowledgebase_path]

def add_metadata(doc, doc_type):
    doc.metadata["doc_type"] = doc_type
    return doc

text_loader_kwargs = {'encoding': 'utf-8'}

# ...existing code...

documents = []
for folder in folders:
    if os.path.isdir(folder):
        doc_type = os.path.basename(folder)
        print(f"Processando pasta: {folder} (tipo: {doc_type})")
        
        # Carrega arquivos .txt
        txt_loader = DirectoryLoader(
            folder, glob="**/*.txt", loader_cls=TextLoader, 
            loader_kwargs=text_loader_kwargs, recursive=True
        )
        txt_docs = txt_loader.load()
        
        # Carrega arquivos .docx
        docx_loader = DirectoryLoader(
            folder, glob="**/*.docx", loader_cls=UnstructuredFileLoader, 
            recursive=True
        )
        docx_docs = docx_loader.load()
        
        # Converte todos para texto puro
        all_docs = txt_docs + docx_docs
        for doc in all_docs:
            # Garante que o conteúdo é texto puro
            text = doc.page_content if hasattr(doc, "page_content") else str(doc)
            # Cria novo Document só com texto puro e metadados
            new_doc = Document(page_content=text, metadata=doc.metadata)
            new_doc.metadata["doc_type"] = doc_type
            documents.append(new_doc)
        print(f"Encontrados {len(all_docs)} documentos nesta pasta")

# ...existing code...

if not documents:
    print("Nenhum documento foi carregado. Verifique:")
    print(f"1. O caminho {knowledgebase_path} existe?")
    print("2. Existem arquivos .txt no diretório ou subdiretórios?")
else:
    text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    chunks = text_splitter.split_documents(documents)
    print(f"Total number of chunks: {len(chunks)}")
    print(f"Document types found: {set(doc.metadata['doc_type'] for doc in documents)}")

In [ ]:
# Create embeddings using OpenAI's text-embedding-3-small model
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# Delete if already exists

if os.path.exists(db_name):
    Chroma(persist_directory=db_name, embedding_function=embeddings).delete_collection()

# Create vectorstore

vectorstore = Chroma.from_documents(documents=chunks, embedding=embeddings, persist_directory=db_name)
print(f"Vectorstore created with {vectorstore._collection.count()} documents")

In [ ]:
# Let's investigate the vectors

collection = vectorstore._collection
count = collection.count()

sample_embedding = collection.get(limit=1, include=["embeddings"])["embeddings"][0]
dimensions = len(sample_embedding)
print(f"There are {count:,} vectors with {dimensions:,} dimensions in the vector store")

In [ ]:
# create a new Chat with OpenAI
llm = ChatOpenAI(
    temperature=0.7,
    model_name=MODEL
)

# Alternative - if you'd like to use Ollama locally, uncomment this line instead
# llm = ChatOpenAI(temperature=0.7, model_name='llama3.2', base_url='http://localhost:11434/v1', api_key='ollama')

# set up the conversation memory for the chat
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)

# the retriever is an abstraction over the VectorStore that will be used during RAG
retriever = vectorstore.as_retriever()

# putting it together: set up the conversation chain with the GPT 3.5 LLM, the vector store and memory
conversation_chain = ConversationalRetrievalChain.from_llm(llm=llm, retriever=retriever, memory=memory)

In [ ]:
sample_questions = [
    # 📝 Atas e Reuniões
    "Quando ocorreu a última assembleia ou reunião registrada do condomínio?",
    "Liste as datas das cinco assembleias ou reuniões mais recentes, em ordem cronológica.",
    "Quais foram os tópicos principais discutidos na assembleia mais recente do condomínio?",
    "Quem são os atuais membros do conselho fiscal, conforme os documentos mais recentes?",
    "De acordo com a última ata de assembleia, qual é o valor atual da taxa condominial definida para os condôminos?",

    # 📄 Contratos
    "Quais contratos ativos o condomínio possui atualmente e com quais empresas foram firmados?",
    "Qual é o valor mensal acordado no contrato de prestação de serviços manutenção de elevadores?",
    "Há cláusulas com penalidades previstas para rescisão antecipada de algum contrato? Se sim, quais?",
    "Quando foi firmado o contrato mais recente e qual é o prazo de vigência previsto?",
    "Existe algum contrato relacionado à manutenção predial, elevadores ou segurança eletrônica? Qual o conteúdo principal?"
]

# 9. Execução das perguntas
for question in sample_questions:
    result = conversation_chain.invoke({"question": question})
    print(f"❓ Pergunta: {question}\n✅ Resposta: {result['answer']}\n")

In [ ]:
# set up a new conversation memory for the chat
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)

# putting it together: set up the conversation chain with the GPT 4o-mini LLM, the vector store and memory
conversation_chain = ConversationalRetrievalChain.from_llm(llm=llm, retriever=retriever, memory=memory)

## Now we will bring this up in Gradio using the Chat interface -

A quick and easy way to prototype a chat with an LLM

In [ ]:
# Wrapping that in a function

def chat(question, history):
    result = conversation_chain.invoke({"question": question})
    return result["answer"]

In [ ]:
# And in Gradio:

view = gr.ChatInterface(chat, type="messages").launch(inbrowser=True)

In [ ]:
# create a new Chat with OpenAI
llm = ChatOpenAI(temperature=0.7, model_name=MODEL)

# set up the conversation memory for the chat
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)

# the retriever is an abstraction over the VectorStore that will be used during RAG; k is how many chunks to use
retriever = vectorstore.as_retriever(search_kwargs={"k": 25})

# putting it together: set up the conversation chain with the GPT 3.5 LLM, the vector store and memory
conversation_chain = ConversationalRetrievalChain.from_llm(llm=llm, retriever=retriever, memory=memory)

In [ ]:
def chat(question, history):
    result = conversation_chain.invoke({"question": question})
    return result["answer"]

In [ ]:
view = gr.ChatInterface(chat, type="messages").launch(inbrowser=True)